In [28]:
# TO DO
# Put this notebook in a utils.py "funcion load model data set"
# Train Regression Model for Spread and Classification Model for Straight Pick Probabilities

In [1]:
import pandas as pd
import numpy as np
import featuretools as ft
import dask.dataframe as dd
from pathlib import Path
from IPython.display import display, HTML
 
# Notebook Options
pd.options.display.max_columns = 20
filepath= Path('data/')

In [2]:
# Games
usecols = ['id', 'season', 'week', 'season_type', 'start_date', 'neutral_site', 'conference_game', 'attendance', 'venue_id',
 'home_id', 'home_team', 'home_conference', 'home_points', 'home_line_scores', 'away_id', 'away_team', 'away_conference', 'away_points', 'away_line_scores',]

dfs = []
for file in filepath.rglob('games.csv'):

    df = pd.read_csv(file, usecols=usecols, parse_dates=['start_date'])
    dfs.append(df)

games_df = pd.concat(dfs, axis=0, ignore_index=True)

games_df['actual_rolling_spread'] =  games_df.away_points - games_df.home_points
games_df['actual_rolling_overunder'] = games_df.home_points + games_df.away_points 
games_df = games_df.rename(columns=dict(id='game_id'))
games_df = games_df.rename(columns=dict(start_date='game_date'))

# Venues 
usecols = ['id', 'name', 'capacity', 'grass', 'city', 'state', 'elevation', 'year_constructed', 'dome', 'timezone']

venues_df = pd.read_csv(filepath/'venues.csv', usecols=usecols)
venues_df = venues_df.rename(columns=dict(id='venue_id'))

# Convert to Bool Columns
venues_df['grass'] = venues_df['grass'].astype('bool')
venues_df['dome'] = venues_df['dome'].astype('bool')

games_df = games_df.merge(venues_df[['venue_id', 'capacity', 'dome', 'elevation', 'grass']], how='left', on='venue_id', validate='many_to_one')
# calculate capacity %
games_df['capacity_perc'] = games_df['attendance'] / games_df['capacity'] 
games_df = games_df.drop('capacity', axis=1)

# Split out Quarter Scores
games_df.home_line_scores = games_df.home_line_scores.astype(str).str.replace('[', '').str.replace(']', '').str.replace('nan', '0,0,0,0')
games_df.away_line_scores = games_df.away_line_scores.astype(str).str.replace('[', '').str.replace(']', '').str.replace('nan', '0,0,0,0')

columns = ['first_qtr_score', 'second_qtr_score', 'third_qtr_score', 'fourth_qtr_score']

games_df[[f'home_{col}' for col in columns]] = games_df.home_line_scores.str.split(',', expand=True).iloc[:, :4].replace('', np.nan).fillna(0).astype(int)
games_df[[f'away_{col}' for col in columns]] = games_df.away_line_scores.str.split(',', expand=True).iloc[:, :4].replace('', np.nan).fillna(0).astype(int)

games_df = games_df.drop(['home_line_scores', 'away_line_scores'], axis=1)

# Reorder
movecols = ['game_id', 'game_date']
games_df = games_df[movecols + [col for col in games_df.columns if col not in movecols ]]

# Normalize Dates
# games_df['game_date'] = games_df.game_date.dt.normalize()

# Game Stats
dropcols = ['points_scored']

dfs = []
for file in filepath.rglob('games_stats_alt.csv'):

    df = pd.read_csv(file)
    dfs.append(df)

games_stats_df = pd.concat(dfs, axis=0, ignore_index=True)
games_stats_df = games_stats_df.drop(dropcols, axis=1)
games_stats_df = games_stats_df.rename(columns=dict(school='school_id'))

# Advanced Stats
dropcols = ['week', 'opponent']

dfs = []
for file in filepath.rglob('games_advanced_stats.csv'):

    df = pd.read_csv(file)
    dfs.append(df)

games_adv_stats_df = pd.concat(dfs, axis=0, ignore_index=True)
games_adv_stats_df = games_adv_stats_df.drop(dropcols, axis=1)
games_adv_stats_df = games_adv_stats_df.rename(columns=dict(team='school_id', gameId='game_id'))
games_adv_stats_df = games_adv_stats_df.fillna(0)

# Merge Stats
games_stats_df = games_stats_df.merge(games_adv_stats_df, on=['game_id', 'school_id'],validate='one_to_one')

# Add Game Date for Rolling Calculations
games_stats_df = games_stats_df.merge(games_df[['game_id', 'game_date']], on='game_id')

# Reorder
movecols = ['game_id', 'school_id', 'home_away', 'game_date', 'season']
games_stats_df = games_stats_df[movecols + [col for col in games_stats_df.columns if col not in movecols ]]
games_stats_df = games_stats_df.fillna(0)

# Need to convert school_id to categorical for featuretools cutofftime
games_stats_df['school_id'] = games_stats_df.school_id.astype('category')
categorical_map = dict(zip(games_stats_df.school_id.cat.codes, games_stats_df.school_id))

# Seperate Stats into Home and Away
games_stats_home_df = games_stats_df[games_stats_df['home_away'] == 'home'].drop('home_away', axis=1)
# games_stats_home_df.columns = games_stats_home_df.columns.map(lambda x : x +'_home' if x not in ['game_id', 'school_id', 'game_date', 'season'] else x)

games_stats_away_df = games_stats_df[games_stats_df['home_away'] == 'away'].drop('home_away', axis=1)
# games_stats_away_df.columns = games_stats_away_df.columns.map(lambda x : x +'_away' if x not in ['game_id', 'school_id', 'game_date', 'season'] else x)

# Merge games_df points for rolling feature calculation

home_cols = ['game_id', 'home_points', 'actual_rolling_spread', 'actual_rolling_overunder', 'home_first_qtr_score',
             'home_second_qtr_score', 'home_third_qtr_score', 'home_fourth_qtr_score']

away_cols = ['game_id', 'away_points', 'away_first_qtr_score','away_second_qtr_score', 
             'away_third_qtr_score', 'away_fourth_qtr_score']

games_stats_home_df = games_stats_home_df.merge(games_df[home_cols], on='game_id', how='left', validate='one_to_one')

games_stats_home_df.rename(columns=dict(
    home_points='points', 
    home_first_qtr_score='first_qtr_score',
    home_second_qtr_score='second_qtr_score',
    home_third_qtr_score='third_qtr_score',
    home_fourth_qtr_score='fourth_qtr_score')
    )

games_stats_away_df = games_stats_away_df.merge(games_df[away_cols], on='game_id', how='left', validate='one_to_one')

games_stats_away_df.rename(columns=dict(
    away_points='points', 
    away_first_qtr_score='first_qtr_score',
    away_second_qtr_score='second_qtr_score',
    away_third_qtr_score='third_qtr_score',
    away_fourth_qtr_score='fourth_qtr_score')
    )

# Drop Columns not needed in games_df
drop_cols = ['home_points', 'home_first_qtr_score', 'home_second_qtr_score', 'home_third_qtr_score', 'home_fourth_qtr_score',  
             'away_points', 'away_first_qtr_score','away_second_qtr_score', 'away_third_qtr_score', 'away_fourth_qtr_score',
             'actual_rolling_spread', 'actual_rolling_overunder']

games_df = games_df.drop(drop_cols, axis=1)

games_df.head()


,game_id,game_date,season,week,season_type,neutral_site,conference_game,attendance,venue_id,home_id,home_team,home_conference,away_id,away_team,away_conference,dome,elevation,grass,capacity_perc
0,400603840,2015-09-03 18:00:00+00:00,2015,1,regular,True,False,51664.0,3628,2579,South Carolina,SEC,153,North Carolina,ACC,False,220.877747,True,0.685090
1,400763593,2015-09-03 18:00:00+00:00,2015,1,regular,False,False,39184.0,3652,2116,UCF,American Athletic,2229,Florida International,Conference USA,False,20.941137,False,0.886396
2,400763399,2015-09-03 19:00:00+00:00,2015,1,regular,False,False,19717.0,3786,2117,Central Michigan,Mid-American,197,Oklahoma State,Big 12,False,240.900772,False,0.599574
3,400756896,2015-09-03 19:00:00+00:00,2015,1,regular,False,False,27126.0,3630,154,Wake Forest,ACC,2210,Elon,NaN,False,283.548584,False,0.861143
4,400787299,2015-09-03 19:00:00+00:00,2015,1,regular,False,False,10473.0,3919,2050,Ball State,Mid-American,2678,VMI,NaN,False,285.481781,False,0.465467


In [3]:
# Teams and Season (Inlcudes Talent and Recruiting Teams)

# Recruiting Teams
dropcols = ['year', 'team']

dfs = []
for file in filepath.rglob('recruiting_teams.csv'):

    df = pd.read_csv(file)
    dfs.append(df)

recruiting_team_df = pd.concat(dfs, axis=0, ignore_index=True)
recruiting_team_df['season_team_id'] = recruiting_team_df['year'].astype(str) + ' - ' + recruiting_team_df['team']
recruiting_team_df = recruiting_team_df.drop(dropcols, axis=1)
recruiting_team_df = recruiting_team_df.rename(columns=dict(rank='recruiting_rank', points='recruiting_points'))

# Talent
dfs = []
for file in filepath.rglob('talent.csv'):

    df = pd.read_csv(file)
    dfs.append(df)

talent_df = pd.concat(dfs, axis=0, ignore_index=True)
talent_df = talent_df.rename(columns=dict(talent='talent_score'))
talent_df['season_team_id'] = talent_df['year'].astype(str) + ' - ' + talent_df['school']
talent_df = talent_df.drop(['year', 'school'], axis=1)
talent_df = talent_df.drop_duplicates(subset=['season_team_id'])

# Teams and Season
team_season_df = games_df[['season', 'home_team', 'away_team']]

team_season_df = pd.concat([
    team_season_df[['season', 'home_team']].rename(columns=dict(home_team='school_id')), 
    team_season_df[['season', 'away_team']].rename(columns=dict(away_team='school_id'))
    ]).drop_duplicates()

team_season_df['season_team_id'] = team_season_df['season'].astype(str) + ' - ' + team_season_df['school_id']

# Merge Tables
team_season_df = team_season_df.merge(recruiting_team_df, on='season_team_id', validate='one_to_one')
team_season_df.head()
team_season_df = team_season_df.merge(talent_df, on='season_team_id', validate='one_to_one')

# Missing Values
team_season_df[['recruiting_rank', 'recruiting_points', 'talent_score']] = team_season_df[['recruiting_rank', 'recruiting_points', 'talent_score']].fillna(team_season_df[['recruiting_rank', 'recruiting_points', 'talent_score']].mean())

team_season_df.head()

,season,school_id,season_team_id,recruiting_rank,recruiting_points,talent_score
0,2015,South Carolina,2015 - South Carolina,20,230.49,725.72
1,2015,UCF,2015 - UCF,71,160.78,484.54
2,2015,Central Michigan,2015 - Central Michigan,97,123.63,376.81
3,2015,Wake Forest,2015 - Wake Forest,51,176.85,497.48
4,2015,Ball State,2015 - Ball State,105,119.70,349.94


In [4]:
# Roster
usecols = ['id', 'weight', 'height', 'year', 'position', 'home_state', 'team', 'season']

dfs = []
for file in filepath.rglob('roster.csv'):

    df = pd.read_csv(file, usecols=usecols)
    dfs.append(df)

roster_df = pd.concat(dfs, axis=0, ignore_index=True)

# Missing Values
roster_df[['weight', 'height', 'year']] = roster_df[['weight', 'height', 'year']].fillna(roster_df.groupby(['team', 'position', 'year'])[['weight', 'height', 'year']].transform('mean'))
roster_df[['weight', 'height', 'year']] = roster_df[['weight', 'height', 'year']].fillna(roster_df.groupby(['team', 'position'])[['weight', 'height', 'year']].transform('mean'))
roster_df[['weight', 'height', 'year']] = roster_df[['weight', 'height', 'year']].fillna(roster_df.groupby(['position'])[['weight', 'height', 'year']].transform('mean'))
roster_df[['weight', 'height', 'year']] = roster_df[['weight', 'height', 'year']].fillna(roster_df.groupby(['team'])[['weight', 'height', 'year']].transform('mean'))
roster_df[['weight', 'height', 'year']] = roster_df[['weight', 'height', 'year']].fillna(roster_df[['weight', 'height', 'year']].mean())

roster_df = roster_df.rename(columns=dict(id='roster_id', 
                                          weight='roster_weight', 
                                          height='roster_height', 
                                          year='roster_year', 
                                          position='roster_position', 
                                          home_state='roster_home_state')
                                          )

roster_df['roster_id'] = roster_df.index
roster_df['season_team_id'] = roster_df['season'].astype(str) + ' - ' + roster_df['team']
roster_df = roster_df.drop(['season', 'team'], axis=1)

# Reorder
movecols = ['roster_id', 'season_team_id']
roster_df = roster_df[movecols + [col for col in roster_df.columns if col not in movecols ]]


roster_df.head()

,roster_id,season_team_id,roster_weight,roster_height,roster_year,roster_position,roster_home_state
0,0,2015 - Air Force,225.000000,74.0,3.321429,LB,IL
1,1,2015 - Air Force,188.964706,72.0,3.380282,DB,GA
2,2,2015 - Air Force,229.230769,76.0,3.600000,TE,KS
3,3,2015 - Air Force,188.964706,73.0,3.380282,DB,GA
4,4,2015 - Air Force,194.285714,70.0,4.000000,?,TX


In [5]:
# Recruiting Player
dfs = []
for file in filepath.rglob('recruiting_position.csv'):

    df = pd.read_csv(file)#, usecols=usecols)
    dfs.append(df)

recruiting_df = pd.concat(dfs, axis=0, ignore_index=True)

recruiting_df = recruiting_df.drop('conference', axis=1)
recruiting_df['season_team_id'] = recruiting_df['season'].astype(str) + ' - ' + recruiting_df['team']
recruiting_df = recruiting_df.drop(['season', 'team'], axis=1)
recruiting_df['recruiting_id'] = recruiting_df.index

# Reorder
movecols = ['recruiting_id', 'season_team_id']
recruiting_df = recruiting_df[movecols + [col for col in recruiting_df.columns if col not in movecols ]]


recruiting_df.head()

,recruiting_id,season_team_id,positionGroup,averageRating,totalRating,commits,averageStars
0,0,2015 - Air Force,Defensive Back,0.764333,2.2930,3,2.000000
1,1,2015 - Air Force,Defensive Line,0.771833,2.3155,3,2.000000
2,2,2015 - Air Force,Linebacker,0.733900,1.4678,2,2.000000
3,3,2015 - Air Force,Offensive Line,0.771333,2.3140,3,2.000000
4,4,2015 - Air Force,Quarterback,0.792900,2.3787,3,2.333333


In [6]:
# Target
usecols = ['id', 'season', 'week', 'home_points', 'away_points']

dfs = []
for file in filepath.rglob('games.csv'):

    df = pd.read_csv(file, usecols=usecols)
    dfs.append(df)

target_df = pd.concat(dfs, axis=0, ignore_index=True)

target_df['actual_spread'] = target_df.away_points - target_df.home_points 
target_df['actual_overunder'] = target_df.home_points + target_df.away_points 
target_df = target_df.drop(['home_points', 'away_points'], axis=1)
target_df = target_df.rename(columns=dict(id='game_id'))

usecols = ['id', 'provider', 'spread', 'overUnder']

dfs = []
for file in filepath.rglob('lines.csv'):

    df = pd.read_csv(file, usecols=usecols)
    dfs.append(df[df.provider=='consensus'])

lines_df = pd.concat(dfs, axis=0, ignore_index=True)
lines_df = lines_df.drop('provider', axis=1)
lines_df = lines_df.rename(columns=dict(id='game_id', spread='sportsbook_spread', overUnder='sportsbook_overunder'))

# Merge Actual and Lines
target_df = target_df.merge(lines_df, on='game_id', how='inner', validate='one_to_one')

# Create Targets for Classifier
target_df['is_home_covered'] = np.where(target_df.actual_spread.lt(target_df.sportsbook_spread), 1, 0)
target_df['is_over'] = np.where(target_df.actual_overunder.gt(target_df.sportsbook_overunder), 1, 0)
target_df['is_home_winner'] = np.where(target_df.actual_spread.lt(0), 1, 0)

target_df.head()

,game_id,season,week,actual_spread,actual_overunder,sportsbook_spread,sportsbook_overunder,is_home_covered,is_over,is_home_winner
0,400603840,2015,1,-4.0,30.0,-3.5,NaN,1,0,1
1,400763593,2015,1,1.0,29.0,-17.0,NaN,0,0,0
2,400763399,2015,1,11.0,37.0,20.5,NaN,1,0,0
3,400603839,2015,1,2.0,26.0,-17.5,NaN,0,0,0
4,400756883,2015,1,-7.0,41.0,-3.0,NaN,1,0,1


In [7]:
# ETS
home_es = ft.EntitySet(id='home')
away_es = ft.EntitySet(id='away')

# Add Teams|Seasons Entity
home_es = home_es.entity_from_dataframe(
            entity_id='team_season',
            dataframe=team_season_df,
            index='season_team_id',
            variable_types={'school_id': ft.variable_types.Id})
away_es = away_es.entity_from_dataframe(
            entity_id='team_season',
            dataframe=team_season_df,    
            index='season_team_id',
            variable_types={'school_id': ft.variable_types.Id})

# Add Roster Entity
home_es = home_es.entity_from_dataframe(
            entity_id='roster',
            dataframe=roster_df,
            index='roster_id',
            variable_types={
                'roster_position': ft.variable_types.Categorical,
                'roster_home_state': ft.variable_types.Categorical})
away_es = away_es.entity_from_dataframe(
            entity_id='roster',
            dataframe=roster_df,    
            index='roster_id',
            variable_types={
                'roster_position': ft.variable_types.Categorical,
                'roster_home_state': ft.variable_types.Categorical,
                'season_team_id': ft.variable_types.Id})

# Add Recruiting Entity
home_es = home_es.entity_from_dataframe(
            entity_id='recruiting',
            dataframe=recruiting_df,
            index='recruiting_id',
            variable_types={'positionGroup': ft.variable_types.Categorical})
away_es = away_es.entity_from_dataframe(
            entity_id='recruiting',
            dataframe=recruiting_df,    
            index='recruiting_id',
            variable_types={'positionGroup': ft.variable_types.Categorical})

# Add Games Entity
# Seperate Games into Home and Away Games
home_cols = ['game_id', 'season', 'home_team']
aways_cols = ['game_id', 'season', 'away_team']

home_games_df = games_df[home_cols]
away_games_df = games_df[aways_cols]

home_games_df['season_team_id'] = home_games_df['season'].astype(str) + ' - ' + home_games_df['home_team']
away_games_df['season_team_id'] = away_games_df['season'].astype(str) + ' - ' + away_games_df['away_team']

home_games_df = home_games_df.rename(columns=dict(home_team='team'))
away_games_df = away_games_df.rename(columns=dict(away_team='team'))

home_es = home_es.entity_from_dataframe(
            entity_id='games',
            dataframe=home_games_df,
            index='game_id')
away_es = away_es.entity_from_dataframe(
            entity_id='games',
            dataframe=away_games_df,    
            index='game_id')

# Create Relationships
relationships = [
  # parent_entity   parent_variable  child_entity  child_variable
  ('team_season',  'season_team_id', 'recruiting', 'season_team_id'),
  ('team_season',  'season_team_id', 'roster',     'season_team_id'),
  ('team_season',  'season_team_id', 'games',      'season_team_id')
]

# Apply the relationships
for es in [home_es, away_es]:
    for pe, pv, ce, cv in relationships:
        es = es.add_relationship(ft.Relationship(es[pe][pv], es[ce][cv]))

# Add interesting valuees
home_es['recruiting']['positionGroup'].interesting_values = recruiting_df.positionGroup.unique()
away_es['recruiting']['positionGroup'].interesting_values = recruiting_df.positionGroup.unique()

home_es['roster']['roster_position'].interesting_values = roster_df.roster_position.unique()
away_es['roster']['roster_position'].interesting_values = roster_df.roster_position.unique()

features = ["mean", "max", "min", "sum"]

# Home Features
home_teams_season_feat, home_dfs_defs = ft.dfs(entityset=home_es,
                            target_entity='games',
                            agg_primitives=features,
                            where_primitives=features,
                            verbose = True)

# Away Features
away_teams_season_feat, away_dfs_defs = ft.dfs(entityset=away_es,
                            target_entity='games',
                            agg_primitives=features,
                            where_primitives=features,
                            verbose = True)

# Remove low information features
home_teams_season_feat = ft.selection.remove_low_information_features(home_teams_season_feat)
away_teams_season_feat = ft.selection.remove_low_information_features(away_teams_season_feat)

# Align  Features
home_teams_season_feat, away_teams_season_feat = home_teams_season_feat.align(away_teams_season_feat, join='inner', axis=1)

Built 472 features
Elapsed: 00:01 | Progress: 100%|██████████
Built 472 features
Elapsed: 00:01 | Progress: 100%|██████████


In [16]:
# Rolling ETS

# Home & Away
# Remove timezone https://github.com/FeatureLabs/featuretools/issues/498
games_stats_home_df['game_date'] = games_stats_home_df['game_date'].dt.tz_convert(None)
games_stats_away_df['game_date'] = games_stats_away_df['game_date'].dt.tz_convert(None)

# Index must be numeric for cutofftimes
games_stats_home_df['school_id'] = games_stats_home_df.school_id.cat.codes
games_stats_away_df['school_id'] = games_stats_away_df.school_id.cat.codes


# Create Entity Sets
rolling_home_es = ft.EntitySet(id='rolling_home')
rolling_away_es = ft.EntitySet(id='rolling_away')

# Home ES
rolling_home_es = rolling_home_es.entity_from_dataframe(
                            entity_id="game_stats",
                            dataframe=games_stats_home_df,
                            index="game_id",
                            time_index="game_date")

rolling_home_es = rolling_home_es.entity_from_dataframe(
                            entity_id="schools",
                            dataframe=pd.DataFrame(games_stats_home_df['school_id'].drop_duplicates()),
                            index="school_id")

# Away ES
rolling_away_es = rolling_away_es.entity_from_dataframe(
                            entity_id="game_stats",
                            dataframe=games_stats_away_df,
                            index="game_id",
                            time_index="game_date")

rolling_away_es = rolling_away_es.entity_from_dataframe(
                            entity_id="schools",
                            dataframe=pd.DataFrame(games_stats_away_df['school_id'].drop_duplicates()),
                            index="school_id")

# Create Relationships
relationships = [
  # parent_entity   parent_variable  child_entity  child_variable
  ('schools',  'school_id', 'game_stats', 'school_id'),
]

# Apply the relationships
for es in [rolling_home_es, rolling_away_es]:
    for pe, pv, ce, cv in relationships:
        es = es.add_relationship(ft.Relationship(es[pe][pv], es[ce][cv]))

cutoff_home_times = pd.DataFrame()
cutoff_home_times[['game_id', 'game_date']] = games_stats_home_df[['game_id', 'game_date']]

cutoff_away_times = pd.DataFrame()
cutoff_away_times[['game_id', 'game_date']] = games_stats_away_df[['game_id', 'game_date']]

features = ["mean", "max", "min"]

# Home Features
home_rolling_feat, rolling_home_dfs_defs = ft.dfs(
                            entityset=rolling_home_es,
                            target_entity='game_stats',
                            agg_primitives=features,
                            cutoff_time=cutoff_home_times,
                            training_window='2 months',
                            cutoff_time_in_index=True,
                            verbose = True)

# Away Features
away_rolling_feat, rolling_away_dfs_defs = ft.dfs(
                            entityset=rolling_away_es,
                            target_entity='game_stats',
                            agg_primitives=features,
                            cutoff_time=cutoff_away_times,
                            training_window='2 months',
                            cutoff_time_in_index=True,
                            verbose = True)


# Remove low information features
home_rolling_feat = ft.selection.remove_low_information_features(home_rolling_feat)
away_rolling_feat = ft.selection.remove_low_information_features(away_rolling_feat)

# Align  Features
home_rolling_feat, away_rolling_feat = home_rolling_feat.align(away_rolling_feat, join='inner', axis=1)

home_rolling_feat.to_hdf('home_rolling_feat.h5', key='home_rolling_feat', mode='w')
away_rolling_feat.to_hdf('away_rolling_feat.h5', key='away_rolling_feat', mode='w')

Built 429 features
Elapsed: 11:05 | Progress: 100%|██████████
Built 421 features
Elapsed: 10:30 | Progress: 100%|██████████


In [8]:
# LOAD TO SKIP ABOVE
home_rolling_feat = pd.read_hdf('home_rolling_feat.h5')
away_rolling_feat = pd.read_hdf('away_rolling_feat.h5')

In [9]:
# Clean Up Feature Tools Dataframes

#################### Team_Season ##########################
home_teams_season_feat = home_teams_season_feat.reset_index()
home_teams_season_feat = home_teams_season_feat.drop(['season_team_id', 'team_season.season', 'team_season.school_id'], axis=1)
# home_teams_season_feat.columns = home_teams_season_feat.columns.map(lambda x : x +'_home' if x not in ['game_id', 'season', 'team'] else x)

away_teams_season_feat = away_teams_season_feat.reset_index()
away_teams_season_feat = away_teams_season_feat.drop(['season_team_id', 'team_season.season', 'team_season.school_id'], axis=1)
# away_teams_season_feat.columns = away_teams_season_feat.columns.map(lambda x : x +'_away' if x not in ['game_id', 'season', 'team'] else x)

###################### Rolling ############################
drop_cols = ['DAY(game_date)', 'MONTH(game_date)', 'WEEKDAY(game_date)', 'YEAR(game_date)', 'time']

home_rolling_feat = home_rolling_feat.reset_index().drop(drop_cols, axis=1)
home_rolling_feat['school_id'] = home_rolling_feat.school_id.map(categorical_map)

away_rolling_feat = away_rolling_feat.reset_index().drop(drop_cols, axis=1)
away_rolling_feat['school_id'] = away_rolling_feat.school_id.map(categorical_map)


In [10]:
# Prep for Model

###################### Games ############################
# Fill Missing Values
games_df['capacity_perc'] = np.where(np.isinf(games_df['capacity_perc']), np.nan, games_df['capacity_perc'])
games_df[['attendance' , 'capacity_perc', 'elevation']] = games_df[['attendance' , 'capacity_perc', 'elevation']].fillna(games_df.groupby('venue_id')[['attendance' , 'capacity_perc', 'elevation']].transform('mean'))
games_df[['attendance' , 'capacity_perc', 'elevation']] = games_df[['attendance' , 'capacity_perc', 'elevation']].fillna(games_df[['attendance' , 'capacity_perc', 'elevation']].mean())
games_df[['home_conference', 'away_conference']] = games_df[['home_conference', 'away_conference']].fillna('missing')

# Shift Venue Attendance Forward
games_df[['attendance', 'capacity_perc']] = games_df.groupby(['venue_id'])[['attendance', 'capacity_perc']].shift(1)

#################### Team_Season ##########################
home_cols = home_teams_season_feat.iloc[:,3:].columns
away_cols = away_teams_season_feat.iloc[:,3:].columns

# Home Fill Missing
home_teams_season_feat[home_cols] = home_teams_season_feat[home_cols].fillna(home_teams_season_feat.groupby(['season', 'team'])[home_cols].transform('mean'))
home_teams_season_feat[home_cols] = home_teams_season_feat[home_cols].fillna(home_teams_season_feat.groupby(['team'])[home_cols].transform('mean'))
home_teams_season_feat[home_cols] = home_teams_season_feat[home_cols].fillna(home_teams_season_feat[home_cols].mean())
1
# Away Fill Missing
away_teams_season_feat[away_cols] = away_teams_season_feat[away_cols].fillna(away_teams_season_feat.groupby(['season', 'team'])[away_cols].transform('mean'))
away_teams_season_feat[away_cols] = away_teams_season_feat[away_cols].fillna(away_teams_season_feat.groupby(['team'])[away_cols].transform('mean'))
away_teams_season_feat[away_cols] = away_teams_season_feat[away_cols].fillna(away_teams_season_feat[away_cols].mean())

###################### Rolling ############################
home_cols = home_rolling_feat.iloc[:,3:].columns
away_cols = away_rolling_feat.iloc[:,3:].columns

# Home Fill Missing
home_rolling_feat[home_cols] = home_rolling_feat[home_cols].fillna(home_rolling_feat.groupby(['season', 'school_id'])[home_cols].transform('mean'))
home_rolling_feat[home_cols] = home_rolling_feat[home_cols].fillna(home_rolling_feat.groupby(['school_id'])[home_cols].transform('mean'))
home_rolling_feat[home_cols] = home_rolling_feat[home_cols].fillna(home_rolling_feat[home_cols].mean())

# Shift Game Stats Forward
home_rolling_feat[home_cols] = home_rolling_feat.groupby(['school_id'])[home_cols].shift(1)

# Away Fill Missing
away_rolling_feat[away_cols] = away_rolling_feat[away_cols].fillna(away_rolling_feat.groupby(['season', 'school_id'])[away_cols].transform('mean'))
away_rolling_feat[away_cols] = away_rolling_feat[away_cols].fillna(away_rolling_feat.groupby(['school_id'])[away_cols].transform('mean'))
away_rolling_feat[away_cols] = away_rolling_feat[away_cols].fillna(away_rolling_feat[away_cols].mean())

# Shift Game Stats Forward
away_rolling_feat[away_cols] = away_rolling_feat.groupby(['school_id'])[away_cols].shift(1)

In [11]:
# Add Home and Away Features Together
teams_season_feat_interaction = (home_teams_season_feat.set_index('game_id').iloc[:,2:] - 
                                away_teams_season_feat.set_index('game_id').iloc[:,2:])

rolling_feat_interaction = (home_rolling_feat.set_index('game_id').iloc[:,2:] - 
                            away_rolling_feat.set_index('game_id').iloc[:,2:])

teams_season_feat_interaction = teams_season_feat_interaction.add_suffix('_interaction')
rolling_feat_interaction = rolling_feat_interaction.add_suffix('_interaction')

In [12]:
# Merge Games and Stats
features_df = games_df.merge(
                home_teams_season_feat.drop(['season', 'team'], axis=1), how='left', validate='one_to_one', on='game_id').merge(
                away_teams_season_feat.drop(['season', 'team'], axis=1), how='left', validate='one_to_one', on='game_id').merge(                      
                home_rolling_feat.drop(['school_id', 'season'], axis=1), how='left', validate='one_to_one', on='game_id').merge( 
                away_rolling_feat.drop(['school_id', 'season'], axis=1), how='left', validate='one_to_one', on='game_id').merge(
                teams_season_feat_interaction, how='left', validate='one_to_one', left_on='game_id', right_index=True).merge(    
                rolling_feat_interaction, how='left', validate='one_to_one', left_on='game_id', right_index=True)

features_df.columns = features_df.columns.str.replace('_x', '_home')
features_df.columns = features_df.columns.str.replace('_y', '_away')

In [13]:
# Remove Games will Null Features & Date
features_df = features_df.dropna()
features_df = features_df.drop('game_date', axis=1)

In [14]:
# Set game_id as index for Alignment
features_df = features_df.set_index('game_id')
target_df = target_df.set_index('game_id')

In [56]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV 
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.calibration import CalibratedClassifierCV
from xgboost import XGBRegressor, XGBClassifier

ct = make_column_transformer(
        (OneHotEncoder(handle_unknown='ignore'), ['season_type', 'home_team', 'away_team', 'home_conference', 'away_conference']), 
        remainder='passthrough'
        )

# Align Target with Features
features, target = features_df.align(target_df, join='inner', axis=0)

features_train, features_test = features[features.season < 2019], features[features.season >= 2019]
target_train, target_test =  target[target.season < 2019], target[target.season >= 2019]

# Spread & Over Under Model Fitted with GridSearch
spread_model = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                colsample_bynode=1, colsample_bytree=0.9, gamma=0, gpu_id=-1,
                importance_type='gain', interaction_constraints='',
                learning_rate=0.1, max_delta_step=0, max_depth=5,
                min_child_weight=5, missing=np.nan, monotone_constraints='()',
                n_estimators=5000, n_jobs=0, num_parallel_tree=1, random_state=0,
                reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.9,
                tree_method='exact', validate_parameters=1, verbosity=None)

spread_pipe = make_pipeline(ct, spread_model)
spread_pipe.fit(features_train, target_train.actual_spread)

spread_pred = pd.DataFrame({'Spread Prediction': spread_pipe.predict(features_test)}, index=features_test.index)

In [57]:
np.sqrt(mean_squared_error(target_test.actual_spread, spread_pred))

18.174853302765275

In [46]:
# Align Target with Features
overunder_model = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                colsample_bynode=1, colsample_bytree=0.9, gamma=0, gpu_id=-1,
                importance_type='gain', interaction_constraints='',
                learning_rate=0.1, max_delta_step=0, max_depth=5,
                min_child_weight=5, missing=np.nan, monotone_constraints='()',
                n_estimators=50, n_jobs=0, num_parallel_tree=1, random_state=0,
                reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.9,
                tree_method='exact', validate_parameters=1, verbosity=None)

overunder_pipe = make_pipeline(ct, overunder_model)
overunder_pipe.fit(features_train, target_train.actual_overunder)

overunder_pred = pd.DataFrame({'OverUnder Prediction': overunder_pipe.predict(features_test)}, index=features_test.index)

In [51]:
class_model = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                colsample_bynode=1, colsample_bytree=0.9, gamma=0, gpu_id=-1,
                importance_type='gain', interaction_constraints='',
                learning_rate=0.1, max_delta_step=0, max_depth=3,
                min_child_weight=5, missing=np.nan, monotone_constraints='()',
                n_estimators=50, n_jobs=0, num_parallel_tree=1, random_state=0,
                reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.7,
                tree_method='exact', validate_parameters=1, verbosity=None)


# Calibrate Probabilities
class_model = CalibratedClassifierCV(
                        base_estimator=class_model,
                        method='isotonic'
                        )

class_pipe = make_pipeline(ct, class_model)
class_pipe.fit(features_train, target_train.is_home_winner)

class_pred = pd.DataFrame({'is Home Winner Prediction': class_pipe.predict(features_test)}, index=features_test.index)

class_pred_prob = pd.DataFrame({'Home Win Probability': class_pipe.predict_proba(features_test)[:,1], 'Away Win Probability': class_pipe.predict_proba(features_test)[:,0]}, index=features_test.index)


In [52]:
df = pd.concat([spread_pred, overunder_pred, class_pred, class_pred_prob], axis=1)
df.head()

,Spread Prediction,OverUnder Prediction,is Hoem Winner Prediction,Home Win Probability,Away Win Probability
game_id,,,,,
401110723,-10.772289,53.842068,1,0.879357,0.120643
401114164,3.001189,61.447735,0,0.364316,0.635684
401117855,-20.416349,63.174084,1,0.783928,0.216072
401117854,-7.760563,59.069340,1,0.690394,0.309606
401119254,-15.366954,61.977047,1,0.786283,0.213717


In [55]:
df.join(target_test.drop(['season', 'week'], axis=1)).to_clipboard()

In [21]:
pred = model.predict(ct.fit_transform(features_test))
pred_prob = model.predict_proba(ct.transform(features_test))
pred_prob = np.max(pred_prob, axis=1)
accuracy_score(target_test.is_home_winner, pred)

# Best: 0.5237603305785123

0.7244582043343654

In [24]:
final = pd.concat(
    [target_test.is_home_winner, 
     pd.Series(pred, index=target_test.index, name='prediction'), 
     pd.Series(pred_prob, index=target_test.index, name='probability')
    ], axis=1)

final['result'] = final.is_home_winner == final.prediction

final['bin'] = pd.cut(final.probability, [.50, .55, .60, .70, .80, .90, 1])

final.groupby('bin')['result'].agg(win=(lambda x: x.sum() / x.count()), count=( 'count'))

,win,count
bin,,
"(0.5, 0.55]",0.392157,51
"(0.55, 0.6]",0.487179,39
"(0.6, 0.7]",0.530864,81
"(0.7, 0.8]",0.639344,122
"(0.8, 0.9]",0.653846,156
"(0.9, 1.0]",0.846154,520


In [44]:
# Feature Importance
important_features = pd.Series(data=spread_model.feature_importances_, index=features_train.columns).sort_values(ascending=False)
important_features.head(50)

IndexError: tuple index out of range

In [45]:
ct.fit_transform(features_train)

array([[0.0, 1.0, 0.0, ..., 0.0, -2.2, 3.9999999999999996],
       [0.0, 1.0, 0.0, ..., -1.0, 0.2999999999999998, 3.0],
       [0.0, 1.0, 0.0, ..., -2.0, 0.0, 3.8999999999999995],
       ...,
       [1.0, 0.0, 0.0, ..., 1.0, 1.0999999999999996, 1.1999999999999997],
       [1.0, 0.0, 0.0, ..., -1.0, 1.5, -0.5],
       [1.0, 0.0, 0.0, ..., 0.0, 2.0, 1.1]], dtype=object)

In [ ]:
# # Grid Search
# parameters = { 
#         'objective':['reg:squarederror'],
#         'learning_rate': [0.1], 
#         'max_depth': [3,5,7],
#         'min_child_weight': [1,3,5],
#         'subsample': [0.5, 0.7, 0.9],
#         'colsample_bytree': [0.7, 0.8, 0.9],
#         'n_estimators': [50, 100, 400]
#         }

# model = GridSearchCV(
#         XGBRegressor(),
#         parameters,
#         scoring='neg_mean_squared_error',
#         cv = 7,
#         n_jobs=-1,
#         verbose=True
#         )

# class_pipe.named_steps['gridsearchcv'].best_estimator_